# Recipe Suggestion with Open Source Model

Works with the gemma-2-2b-it model found in Google's huggingface.

The user's request and the recipes filtered from the data set are put into the prompt. At the same time, the output we want and what we pay attention to are explained in this prompt. Accordingly, the model suggests a recipe.

All steps were applied, but since the model works locally, it took a long time to get the result, so tuning could not be done

In [1]:
import pandas as pd
import re
import ast
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

class RecipeGenerator:
    
    def __init__(self, api_key: str, csv_file: str, user_input: str, model_name: str):

        # Hugginf face api keyinin ortam değişkeni olarak atanması
        os.environ["HUGGING_FACE_HUB_TOKEN"] = api_key
        # Kullanıcı girdisi
        self.user_input = user_input
        # Kullanılacak modelin ismi
        self.model_name = model_name

        # Verinin okunması ve preprocess edilmesi
        self.df = self.load_preprocess_recipes(csv_file)

        model, tokenizer = self.load_model_and_tokenizer()
        print("Model and tokenizer loaded successfully!")

        recipe_suggestions = self.get_recipe_suggestions(self.user_input, self.df)

        prompt = f"""User is looking for recipes matching the following criteria: {user_input}

        Please analyze the following recipe options and recommend the best one, explaining your choice. Then, present the chosen recipe in a clear, step-by-step format:

        1. Start with the list of ingredients.
        2. Follow with the preparation instructions, clearly numbered.
        3. If the user inputs anything such as ingredients, fruits, vegetables, etc., these should also be in the recipe. 
        4. Preparation Time: Estimated total preparation and cooking time

        Ensure that any specific ingredients or preferences mentioned in the user input are addressed in your response. Aim for a clear, concise, and informative presentation that a home cook can easily follow.

        Here are the recipe options:

        {recipe_suggestions}"""

        formatted_recipe = self.chat_with_model(prompt, model, tokenizer)
        print("\nFormatted Recipe:")
        print(formatted_recipe)

    def convert_keywords(self, keywords_str):

        """
        Verilen değişkenin formatını stringden listeye çevirir.

        Params:
        --------
        keyword_str (str): formatı değişecek değişken

        Return:
        ---------
        keywords_str (lıst): formatı değişmiş değişken
        """
        # Eğer giriş zaten bir string değilse, olduğu gibi döndür
        if isinstance(keywords_str, str):
            try:
                return ast.literal_eval(keywords_str)
            except:
                return keywords_str.strip('"').split('", "')
        return keywords_str

    def load_preprocess_recipes(self, path):
        """
        Tariflerin olduğu veriyi yükler ve preprocessing işlemlerini yapar

        Params:
        --------
        path (str): verinin dosya konumu

        Return:
        ---------
        df (DataFrame): Düzenlenmiş tarif verisi
        """
        # veri okunur
        df = pd.read_csv(path)
        # kullanılacak sütunlar seçilir
        chosen_columns = ['Name', 'RecipeCategory', 'Keywords', 'RecipeIngredientParts', 'Calories', 'RecipeInstructions']
        # Na veriler boş alan ile doldurulur
        df = df[chosen_columns].fillna('')
        # Kalori değişkeni nümeriğe çevriliyor ve nümerik olmayan değerler NaN ile dolduruluyor
        df['Calories'] = pd.to_numeric(df['Calories'], errors='coerce')

        # Değişkenlerdeki istenmeyen karakterler çıkarılır
        for col in ['Keywords', 'RecipeIngredientParts', 'RecipeInstructions']:
            df[col] = df[col].str.replace("c(", "").str.replace(")", "").apply(self.convert_keywords)

        return df

    def parse_user_input(self, user_input):
        """
        Kullanıcının girdiği input içerisinden kalori, kategori, ve keywordleri ayıklar

        Params:
        ---------
        user_input (str): kullanıcının tarifini istediği girdi

        Return:
        ---------
        min_calories (int): istenilen minimum kalori
        max_calories (int): istenilen maksimum kalori
        category (str): istenilen kategori
        keywords (list): aranan keywordler
        """
        # Kalori aralığını kontrol eder
        calorie_match = re.search(r'(\d+)-(\d+)\s*Calories', user_input, re.IGNORECASE)
        if calorie_match:
            min_calories, max_calories = map(int, calorie_match.groups())
        else:
            # Tek bir kalori değeri varsa onu kontrol eder
            calorie_match = re.search(r'(\d+)\s*Calories', user_input, re.IGNORECASE)
            if calorie_match:
                min_calories = 0
                max_calories = int(calorie_match.group(1))
            else:
                min_calories, max_calories = 0, 1000

        # Kalori bilgisini girdiden çıkarılır ve metin temizlenir
        keywords = re.sub(r'\d+\s*-\s*\d+\s*Calories|\d+\s*Calories', '', user_input, flags=re.IGNORECASE).strip()

        # kalan kelimeler liste halinde boşluklardan ayrılır
        words_list = keywords.split()

        # İlk kelimeyi kategori değeri olarak alır
        category = words_list[0] if words_list else ""

        # Diğer kelimeleri keyword olarak alır
        keywords = words_list[1:] if len(words_list) > 1 else []

        return min_calories, max_calories, category, keywords


    def find_recipes(self, min_calories, max_calories, category, keywords):
        """
        Verilen parametrelere göre veriyi filtreler ve uygun tarifleri bulur

        Params:
        ---------
        min_calories (int): istenilen minimum kalori
        max_calories (int): istenilen maksimum kalori
        category (str): istenilen kategori
        keywords (list): aranan keywordler

        Return:
        ---------
        filtered_df(DataFrame): istenilen değerlere göre filtrelenmiş veri
        """

        # Veri kaloriye göre filtrelenir
        filtered_df = self.df[(self.df['Calories'] >= min_calories) & (self.df['Calories'] <= max_calories)]
        
        # Eğer kategori var ise kategoriye göre filtrelenir
        if category:
            filtered_df = filtered_df[filtered_df['RecipeCategory'].str.contains(category, case=False)]
        
        # Keyword var ise ona göre filtrelenir
        if keywords:
            for keyword in keywords:
                keyword_lower = keyword.lower()
                filtered_df = filtered_df[
                    filtered_df['Name'].str.lower().str.contains(keyword_lower) |
                    filtered_df['Keywords'].apply(lambda kw_list: keyword_lower in [k.lower() for k in kw_list])]
        
        return filtered_df

    def get_recipe_suggestions(self, user_input, df, num_options=5):
        # Kuallıcı girdisi içinden bilgiler alınır
        min_calories, max_calories, category, keywords = self.parse_user_input(user_input)
        
        # Bilgilere göre tarif bulunur
        recipes = self.find_recipes(min_calories, max_calories, category, keywords)
        
        # Eğer tarif bulunamadıysa, kullanıcıya bilgi ver
        if recipes.empty:
            return f"No recipes found for the given input: {user_input}"
        
        # İstenen sayıda tarif seçilir
        if len(recipes) > num_options:
            recipes = recipes.sample(num_options)
        else:
            recipes = recipes.sample(frac=1)
        
        # Seçilen tarifleri formatlanır
        response = "Here are some recipes that match the criteria:\n\n"
        for _, row in recipes.iterrows():
            ingredients = self.convert_keywords(row['RecipeIngredientParts'])
            instructions = self.convert_keywords(row['RecipeInstructions'])
            
            response += f"Recipe: {row['Name']}\n"
            response += f"Calories: {row['Calories']}\n"
            response += "Ingredients:\n"
            for ingredient in ingredients:
                response += f"- {ingredient}\n"
            
            response += "Instructions:\n"
            for i, instruction in enumerate(instructions, 1):
                response += f"{i}. {instruction}\n"
            
            response += f"Keywords: {', '.join(row['Keywords'])}\n\n"
        
        return response

    def load_model_and_tokenizer(self):
        """
        Model ve tokenizer yüklenir

        Return:
        model: Yüklenen model
        tokenizer: yüklenen tokenizer

        """
        # Hugging Face API tokenını al
        token = os.getenv("HUGGING_FACE_HUB_TOKEN")
        # bulunamazsa error döndür
        if not token:
            raise ValueError("Please set the HUGGING_FACE_HUB_TOKEN environment variable")

        # Tokenizerı yükle
        tokenizer = AutoTokenizer.from_pretrained(self.model_name, token=token)
        # Modeli yükle
        model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            token=token,
            torch_dtype=torch.float16,
        )
        return model, tokenizer

    def generate_response(self,model, tokenizer, prompt, max_length=1000):
        """
        Modele girdi gönderilerek model çıktısı üretilir

        Params:
        --------
        model : yüklenen model
        tokenizer : yüklenen tokenizer
        prompt: Modele verilecek olan prompt
        max_length: maksimum çıktı uzunluğu

        Return:
        --------
        response: modelden dönen çıktı
        """
        # Girdiyi tokenize eder ve model girdisine uygun formata dönüştürür
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
        with torch.no_grad():
            # Modeli kullanarak çıktı üret
            outputs = model.generate(**inputs, max_length=max_length, num_return_sequences=1, temperature=0.7)

        #  Üretilen çıktıyı decode eder ve özel tokenleri atlar
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response
    
    def extract_recipe_info(self, text):
        """
        Tarif bilgileri text içerisinden ayıklanır.

        Params:
        --------
        text: modelden çıkan çıktı

        Return:
        ----------
        recipe_info (str): ayıklanan tarif bilgisi
        """

        # Tarif bilgilerini ayıklamak için kullanılacak regex kalıpları
        patterns = {
            "Selected Recipe": r"(?:Selected Recipe:|1\.)\s*(.*?)(?:\n|$)",
            "Ingredients": r"(?:Ingredients:|3\.)(.*?)(?:Instructions|4\.|$)",
            "Instructions": r"(?:Instructions:|4\.)(.*?)(?:Nutritional Summary|5\.|$)",
            "Preparation Time": r"(?:Preparation Time:|6\.)\s*(.*?)(?:\n|$)"
        }
        # Tarif bilgilerini saklamak için boş bir sözlük oluşturur
        recipe_info = {}
        # Her bir bilgi türü için metni tarar
        for key, pattern in patterns.items():
            # Regex ile eşleşme arar
            match = re.search(pattern, text, re.DOTALL | re.IGNORECASE)
            if match:
                # Eşleşen metni alınır ve boşluklar temizlenir
                value = match.group(1).strip()
                # Malzemeler ve Talimatlar için özel işlem uygulanır
                if key in ["Ingredients", "Instructions"]:
                    # Madde işaretleri uygulanır
                    value = [item.strip() for item in re.split(r'\n-|\n\d+\.', value) if item.strip()]
                recipe_info[key] = value
            else:
                # Eşleşme bulunamazsa varsayılan değer atar
                recipe_info[key] = "Information not provided"
        
        return recipe_info
    
    def format_recipe(self, response):
        """
        Modelden dönen çıktının formatlanması

        Params:
        ---------
        response: model çıktısı

        Return:
        ---------
        formatted_output: formatlanan çıktı
        """
        # Tarif bilgisinin alınması
        recipe_info = self.extract_recipe_info(response) 
        
        # Tarif isminin alınması
        recipe_name = recipe_info.get('Selected Recipe') 
        
        # Kalorilerin alınması
        calories_match = re.search(r'Calories: (\d+(?:\.\d+)?)', response)
        calories = calories_match.group(1) if calories_match else "N/A"
        
        # Malzemelerin alınması
        ingredients = recipe_info.get('Ingredients', [])
        if isinstance(ingredients, str):
            ingredients = [ing.strip() for ing in ingredients.split('-') if ing.strip()]
        
        # Tarif yöntemlerinin alınması
        instructions = recipe_info.get('Instructions', [])
        if isinstance(instructions, str):
            instructions = [inst.strip() for inst in instructions.split('\n') if inst.strip()]
        
        # Tarfin formatlanması
        formatted_output = f"Here is the chosen recipe for {recipe_name}:\n\n"
        formatted_output += f"Calories: {calories}\n\n"
        formatted_output += "Ingredients:\n"
        for ingredient in ingredients:
            formatted_output += f"- {ingredient}\n"
        
        formatted_output += "\nPreparation:\n"
        for i, instruction in enumerate(instructions, 1):
            formatted_output += f"{i}. {instruction}\n"
        
        return formatted_output
    
    def chat_with_model(self, prompt, model, tokenizer):
        """
        Modele girdiler göndeirlir ve çıktı alınır
        
        """
        response = self.generate_response(model, tokenizer, prompt)
        formatted_recipe = self.format_recipe(response)
        return formatted_recipe



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
rg = RecipeGenerator(api_key="api_key", csv_file="tarif.csv", 
                    user_input="Chicken Salad with maximum 500 calories", model_name="google/gemma-2-2b-it")

print(rg)